import packages

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from osgeo import gdal_array
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
from torch import Tensor, einsum
from torch import nn
from torch.nn import functional as F
from scipy.ndimage import distance_transform_edt as distance
from scipy.spatial.distance import directed_hausdorff
from typing import Any, Callable, Iterable, List, Set, Tuple, TypeVar, Union
from torch.utils.data import TensorDataset, DataLoader
from torch.autograd import Variable
import matplotlib
import torch.nn.functional as F
from IPython.display import FileLink


In [2]:
!pip install segmentation-models-pytorch==0.1.0
!pip install keras-unet
!pip install pytorch-lighting
 

     |████████████████████████████████| 42 kB 537 kB/s 
     |████████████████████████████████| 58 kB 2.8 MB/s 
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.0-py3-none-any.whl size=16035 sha256=0874d6364868553d89889afe30babedf95ea04ac67e7247ca765ae93d5b3c67d
  Stored in directory: /root/.cache/pip/wheels/b7/cc/0d/41d384b0071c6f46e542aded5f8571700ace4f1eb3f1591c29
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60962 sha256=4b08dcb28898e44f553ff155949331386a534d3f9d13054a1c38f34c2a338493
  Stored in directory: /root/.cache/pip/wheels/ed/27/e8/9543d42de2740d3544db96aefef63bda3f2c1761b3334f4873
Successfully built efficientnet-pytorch pretrainedmodels
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
ERROR: Could not find a version that satisfies the requirement 

In [3]:
import segmentation_models_pytorch as smp
from keras_unet.utils import plot_imgs

import pytorch_lightning as pl

-----------------------------------------
keras-unet init: TF version is >= 2.0.0 - using `tf.keras` instead of `Keras`
-----------------------------------------


In [4]:
size = 32
tr_tiles =2
nr_channels=7
bs = 16 #batchsize
classes = [ 1,  2,   4,  5,  6,  7,  8,  9, 10]
num_class = len(classes)
EPOCH = 300
reduce = 8

Used to crop the imput images

In [5]:
def slice (arr, size, inputsize,stride):
    result = []
    if stride is None:
        stride = size
    for i in range(0, (inputsize-size)+1, stride):
        for j in range(0, (inputsize-size)+1, stride):
        
            if arr.ndim == 3:
                s = arr[:,i:(i+size),j:(j+size), ]
            else:
                s = arr[i:(i+size),j:(j+size), ]
            result.append(s)
            #print(i,"",j)
    result = np.array(result)
    return result

def batchslice (arr, size, inputsize, stride, num_img):
    result = []
    for i in range(0, num_img):
        s= slice(arr[i,], size, inputsize, stride )
        result.append(s )
    result = np.array(result)
    result = result.reshape(result.shape[0]*result.shape[1], result.shape[2], result.shape[3], -1)
    return result

def class2dim (mask, CLASSES):
    
        masks = [(mask == v) for v in CLASSES ]
        mask = np.stack(masks, axis=-1).astype('float')    
        return mask

load the different tiles into 1 variable

In [6]:
#stack all files into 1 variable
def load_raster(path,data,tile1,tile2,reduce):
    tiles_in=[tile1,tile2]
    files = []
    for tile in tiles_in:
        for file in glob.glob(path+data+'*{}.tif'.format(tile)):
            file1 = gdal_array.LoadFile(file)
            #only use 50% of the points to reduce memory
            if np.ndim(file1)==3:
                file1=file1[:,::reduce,::reduce]
            else:
                file1=file1[::reduce,::reduce]
            files.append(file1)
            print(file1.shape)
    stacked = np.array(files)
    return stacked

def load_data(path,data1,data2,data3,tile1,tile2, reduce):
    part1 = load_raster(path,data1,tile1,tile2, reduce )
    part2 = load_raster(path,data2,tile1,tile2,reduce )
    part3 = load_raster(path,data3,tile1,tile2,reduce )
    
    if np.ndim(part1)< np.ndim(part2):#check if dimmensions are equal
        part1 = np.expand_dims(part1,axis=1)
    elif np.ndim(part1)> np.ndim(part2):
        part2 = np.expand_dims(part2,axis=1)
    if np.ndim(part1)< np.ndim(part3):#check if dimmensions are equal
        part1 = np.expand_dims(part1,axis=1)
    elif np.ndim(part1)> np.ndim(part3):
        part3 = np.expand_dims(part3,axis=1)
    print(part1.shape,part2.shape,part3.shape)
    total = np.concatenate((part1,part2,part3),axis=1)
    return total
def load_raster2(path,data,tile1,tile2,tile3,tile4,reduce):
    tiles_in=[tile1,tile2,tile3,tile4]
    files = []
    for tile in tiles_in:
        print(tile)
        for file in glob.glob(path+data+'*{}.tif'.format(tile)):
            file1 = gdal_array.LoadFile(file)
            #only use 50% of the points to reduce memory
            if np.ndim(file1)==3:
                file1=file1[:,::reduce,::reduce]
            else:
                file1=file1[::reduce,::reduce]
            files.append(file1)
            print(file1.shape)
        stacked = np.array(files)
    return stacked

def load_data2(path,data1,data2,data3,tile1,tile2,tile3,tile4, reduce):
    part1 = load_raster2(path,data1,tile1,tile2,tile3,tile4,reduce )
    part2 = load_raster2(path,data2,tile1,tile2,tile3,tile4,reduce )
    part3 = load_raster2(path,data3,tile1,tile2,tile3,tile4,reduce )
    
    if np.ndim(part1)< np.ndim(part2):#check if dimmensions are equal
        part1 = np.expand_dims(part1,axis=1)
    elif np.ndim(part1)> np.ndim(part2):
        part2 = np.expand_dims(part2,axis=1)
    if np.ndim(part1)< np.ndim(part3):#check if dimmensions are equal
        part1 = np.expand_dims(part1,axis=1)
    elif np.ndim(part1)> np.ndim(part3):
        part3 = np.expand_dims(part3,axis=1)
    print(part1.shape,part2.shape,part3.shape)
    total = np.concatenate((part1,part2,part3),axis=1)
    return total
def load_raster3(path,data,tile1,tile2,tile3,tile4,tile5,tile6,tile7,tile8,reduce):
    tiles_in=[tile1,tile2,tile3,tile4,tile5,tile6,tile7,tile8]
    files = []
    for tile in tiles_in:
        print(tile)
        for file in glob.glob(path+data+'*{}.tif'.format(tile)):
            file1 = gdal_array.LoadFile(file)
           #only use 50% of the points to reduce memory
            if np.ndim(file1)==3:
                file1=file1[:,::reduce,::reduce]
            else:
                file1=file1[::reduce,::reduce]
            files.append(file1)
            print(file1.shape)
        stacked = np.array(files)
    return stacked

def load_data3(path,data1,data2,data3,tile1,tile2,tile3,tile4,tile5,tile6,tile7,tile8 ,reduce):
    part1 = load_raster3(path,data1,tile1,tile2,tile3,tile4,tile5,tile6,tile7,tile8,reduce)
    part2 = load_raster3(path,data2,tile1,tile2,tile3,tile4,tile5,tile6,tile7,tile8,reduce)
    part3 = load_raster3(path,data3,tile1,tile2,tile3,tile4,tile5,tile6,tile7,tile8,reduce)
    
    if np.ndim(part1)< np.ndim(part2):#check if dimmensions are equal
        part1 = np.expand_dims(part1,axis=1)
    elif np.ndim(part1)> np.ndim(part2):
        part2 = np.expand_dims(part2,axis=1)
    if np.ndim(part1)< np.ndim(part3):#check if dimmensions are equal
        part1 = np.expand_dims(part1,axis=1)
    elif np.ndim(part1)> np.ndim(part3):
        part3 = np.expand_dims(part3,axis=1)
    print(part1.shape,part2.shape,part3.shape)
    total = np.concatenate((part1,part2,part3),axis=1)
    return total

In [7]:
if tr_tiles == 2:
    x_train0= load_data("../input/guided-research/Training/","DEM","Wadden","slope","12_7","13_8",reduce)
    x_val0 = load_data("../input/guided-research/Training/","DEM","Wadden","slope","12_8","14_7",reduce)
    y_train0= load_raster("../input/guided-research/Training/","class","12_7","13_8", reduce )
    y_val0=load_raster("../input/guided-research/Training/","class","12_8","14_7",reduce)
if tr_tiles == 4:
    x_train0= load_data2("../input/guided-research/Training/","DEM","Wadden","slope","12_7","12_8","13_8","14_7",reduce)
    x_val0 = load_data("../input/guided-research/Validation/","DEM","Wadden","slope","15_7","15_8",reduce)
    y_train0= load_raster2("../input/guided-research/Training/","class","12_7","12_8","13_8","14_7", reduce )
    y_val0=load_raster("../input/guided-research/Validation/","class","15_7","15_8",reduce)
if tr_tiles == 8:
    x_train0= load_data3("../input/guided-research/Training/","DEM","Wadden","slope","12_7","12_8","13_6","13_7","13_8","13_9","14_7","15_7",reduce) 
    x_val0 = load_data("../input/guided-research/Validation/","DEM","Wadden","slope","15_7","15_8",reduce) 
    y_train0= load_raster3("../input/guided-research/Training/","class","12_7","12_8","13_6","13_7","13_8","13_9","14_7","15_7",reduce )
    y_val0=load_raster("../input/guided-research/Validation/","class","15_7","15_8",reduce )    

(2000, 2000)
(2000, 2000)
(3, 2000, 2000)
(3, 2000, 2000)
(2000, 2000)
(2000, 2000)
(2, 1, 2000, 2000) (2, 3, 2000, 2000) (2, 1, 2000, 2000)
(2000, 2000)
(2000, 2000)
(3, 2000, 2000)
(3, 2000, 2000)
(2000, 2000)
(2000, 2000)
(2, 1, 2000, 2000) (2, 3, 2000, 2000) (2, 1, 2000, 2000)
(2000, 2000)
(2000, 2000)
(2000, 2000)
(2000, 2000)


In [8]:
def ndvi(input1):
    im =np.zeros(shape=(0,2000,2000))
    for i in range(input1.shape[0]):
        nir = input1[i,1,:,:]
        vis = input1[i,2,:,:]
        ndvi = (nir-vis)/(nir+vis)
        ndvi=np.expand_dims(ndvi,axis=0)
        im = np.concatenate([im,ndvi],axis=0)
    im = np.expand_dims(im,axis=1)
    outp = np.concatenate([input1,im],axis=1)
    return outp

In [9]:
x_train0 = ndvi(x_train0)
x_val0 = ndvi(x_val0)
print(x_train0.shape,x_val0.shape,y_train0.shape,y_val0.shape)

(2, 6, 2000, 2000) (2, 6, 2000, 2000) (2, 2000, 2000) (2, 2000, 2000)


In [10]:
def brightness(input1):
    im =np.zeros(shape=(0,2000,2000))
    for i in range(input1.shape[0]):
        nir = input1[i,1,:,:]
        red = input1[i,2,:,:]
        green = input1[i,3,:,:]
        brightness = (nir+red+green)/3
        brightness=np.expand_dims(brightness,axis=0)
        im = np.concatenate([im,brightness],axis=0)
    im = np.expand_dims(im,axis=1)
    outp = np.concatenate([input1,im],axis=1)
    return outp

In [11]:
x_train0 = brightness(x_train0)
x_val0 = brightness(x_val0)
print(x_train0.shape,x_val0.shape,y_train0.shape,y_val0.shape)

(2, 7, 2000, 2000) (2, 7, 2000, 2000) (2, 2000, 2000) (2, 2000, 2000)


In [12]:
def process(x_train, x_val, y_train, y_val,  size, bslice=True, cl2dim=True, Inf2zero=True):
    if bslice :
        x_train  = batchslice(x_train, size,x_train[0].shape[1],size, x_train.shape[0])
        x_val = batchslice(x_val,size, x_val[0].shape[1], size, x_val.shape[0])
        y_train = batchslice(y_train,size,y_train[0].shape[1],size,y_train.shape[0]).squeeze()
        y_val = batchslice(y_val,size,y_val[0].shape[1],size,y_val.shape[0]).squeeze()
        print(f"batch slice to {size}")
       
    if  cl2dim :    
        y_train = class2dim(y_train, classes)
        y_val = class2dim(y_val, classes)

        y_train=  np.moveaxis(y_train, -1, 1)
        y_val = np.moveaxis(y_val, -1, 1)
        print("classes are converted to channels")
        
    if Inf2zero :
        remove = np.unique(x_train[0,4,:,:])
        x_train[np.isinf(x_train)] = 0 
        x_train[np.isnan(x_train)] = 0
        x_train[x_train == remove[0]] = 0
        #np.nan
        x_val [np.isinf(x_val)] =0
        x_val[np.isnan(x_val)] = 0 
        x_val[x_val == remove[0]] = 0
        y_train[np.isinf(y_train)] = 0
        y_train[np.isnan(y_train)] = 0 
        y_val[np.isinf(y_val)] = 0
        y_val[np.isnan(y_val)] = 0 
        
        print("inf are converted to 0 and nan to 0")
    return(x_train, x_val, y_train, y_val)

def myloader(trainX, trainY, valX,valY):

    train = TensorDataset(torch.Tensor(x_train[:,:,:,:]), torch.Tensor(y_train )) # create your datset
    train  = DataLoader(train, batch_size=bs) # create your dataloader
    
    vali= TensorDataset(torch.Tensor(x_val[:,:,:,:]),torch.Tensor(y_val   )) # create your datset
    vali = DataLoader(vali, batch_size=bs) # create your dataloader
    return train , vali 

In [13]:
GAMMA = 2
ALPHA = 0.8 # emphasize FP
BETA = 0.2 # more emphasize on FN

# combo loss
cl_ALPHA = 0.5 # < 0.5 penalises FP more, > 0.5 penalises FN more
cl_BETA = 0.5
CE_RATIO = 0.5 #weighted contribution of modified CE loss compared to Dice loss
e=1e-07

In [14]:
class DiceBCELoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceBCELoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):
        
        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = F.sigmoid(inputs)       
        
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        intersection = (inputs * targets).sum()                            
        dice_loss = 1 - (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)  
        BCE = F.binary_cross_entropy(inputs, targets, reduction='mean')
        Dice_BCE = BCE + dice_loss
        
        return Dice_BCE


     
class IoULoss(nn.Module):
    def __init__(self, weight=None, size_average=True, **kwargs):
        super(IoULoss, self).__init__(**kwargs)

    def forward(self, inputs, targets, smooth=1):
        
        #comment out if your model contains a sigmoid or equivalent activation layer
        #inputs = F.sigmoid(inputs)       
        #print(inputs.shape)
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        #inputs = (inputs>0.5).float()
        #intersection is equivalent to True Positive count
        #union is the mutually inclusive area of all labels & predictions 
        intersection = (inputs * targets).sum()  
        total = (inputs + targets).sum()
        union = total - intersection 
        
        IoU = (intersection + smooth)/(union + smooth) 
                
        return 1 - IoU
    
class TverskyLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(TverskyLoss, self).__init__()

    def forward(self, inputs, targets, smooth=1, alpha=ALPHA, beta=BETA):
        
        #comment out if your model contains a sigmoid or equivalent activation layer
        #inputs = F.sigmoid(inputs)       
        
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        #True Positives, False Positives & False Negatives
        TP = (inputs * targets).sum()    
        FP = ((1-targets) * inputs).sum()
        FN = (targets * (1-inputs)).sum()
       
        Tversky = (TP + smooth) / (TP + alpha*FP + beta*FN + smooth)  
        
        return 1 - Tversky
 

class FocalTverskyLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(FocalTverskyLoss, self).__init__()

    def forward(self, inputs, targets, smooth=1, alpha=ALPHA, beta=BETA, gamma=GAMMA):
        
        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = F.sigmoid(inputs)       
        
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        #True Positives, False Positives & False Negatives
        TP = (inputs * targets).sum()    
        FP = ((1-targets) * inputs).sum()
        FN = (targets * (1-inputs)).sum()
        
        Tversky = (TP + smooth) / (TP + alpha*FP + beta*FN + smooth)  
        FocalTversky = (1 - Tversky)**gamma
                       
        return FocalTversky

class myLoss(torch.nn.Module):

    def __init__(self, pos_weight=1):
      super().__init__()
      self.pos_weight = pos_weight

    def forward(self, input, target):
      epsilon = 10 ** -44
      input = input.sigmoid().clamp(epsilon, 1 - epsilon)

      my_bce_loss = -1 * (self.pos_weight * target * torch.log(input)
                          + (1 - target) * torch.log(1 - input))
      add_loss = (target - 0.5) ** 2 * 4
      mean_loss = (my_bce_loss * add_loss).mean()
      return mean_loss
    
 

class FocalLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(FocalLoss, self).__init__()

    def forward(self, inputs, targets, alpha=ALPHA, gamma=GAMMA, smooth=1):
        
        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = F.sigmoid(inputs)       
        
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        #first compute binary cross-entropy 
        BCE = F.binary_cross_entropy(inputs, targets, reduction='mean')
        BCE_EXP = torch.exp(-BCE)
        focal_loss = alpha * (1-BCE_EXP)**gamma * BCE
                       
        return focal_loss

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [16]:
class CNNmodel(pl.LightningModule):
    def __init__(self):
        super(CNNmodel,self).__init__()
        self.batch_size=bs
        self.learning_rate = 2e-4
        self.net = smp.Unet(classes=num_class, in_channels=7, activation = 'softmax')
        self.label_type = torch.float32 if num_class  == 1 else torch.long
    
    def forward(self,x):
        return self.net(x)
    
    def training_step(self,train_batch,batch_nb):
        x,y = train_batch
        y = y.float()
        y_hat = self(x)
        loss1=IoULoss()
        loss = loss1(y_hat, y)
        return{'loss':loss}
    
    def validation_step(self,val_batch,batch_nb):
        x,y = val_batch
        y = y.long()
        y_hat = self(x)
        loss1=IoULoss()
        val_loss = loss1(y_hat, y)
        return {'val_loss':val_loss}
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=2e-4)
        scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[15, 20, 40], gamma=0.3)
        return [optimizer],[scheduler]
    
    def train_dataloader(self):
        return train_loader
    
    def valid_dataloader(self):
        return valid_loader

In [17]:
from pytorch_lightning.callbacks import ModelCheckpoint

In [18]:
x_train, x_val, y_train, y_val = process(x_train0, x_val0, y_train0, y_val0, size =size, bslice=False, cl2dim=True, Inf2zero=True)
x_train, x_val, y_train, y_val = process(x_train0, x_val0, y_train0, y_val0, size =size, bslice=True, cl2dim=True, Inf2zero=True)
train_loader, valid_loader = myloader(x_train, y_train, x_val, y_val)

classes are converted to channels
inf are converted to 0 and nan to 0
batch slice to 32
classes are converted to channels
inf are converted to 0 and nan to 0


In [19]:
model = CNNmodel()
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    dirpath="/kaggle/working/output/",
    filename="s_%s_%s-{epoch:02d}_%s"%(tr_tiles,size,nr_channels),
)
trainer = pl.Trainer(gpus=1,max_epochs = EPOCH,callbacks=[checkpoint_callback])
modelt = trainer.fit(model,train_loader,valid_loader)

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type | Params
------------------------------
0 | net  | Unet | 24.5 M


/opt/conda/lib/python3.7/site-packages/segmentation_models_pytorch/base/modules.py:89: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self.activation(x)


In [20]:
print(checkpoint_callback.best_model_path,checkpoint_callback.best_model_score)
FileLink(checkpoint_callback.best_model_path[16:])

/kaggle/working/output/s_2_32-epoch=06_7.ckpt tensor(0.0021, device='cuda:0')


/kaggle/working/output/s_2_32-epoch=06_7.ckpt

after download changes the name to s_(#traingin_tiles)_(size)